## Recomendación por filtrado colabarativo

Como ya sabemos, las recomendaciones por filtrado colaborativo, usan la propia información que los usuarios nos deja al interactuar con nuestros items. Pordemos ver la idea en el siguiente gif:

![](../img/450px-Collaborative_filtering.gif)

Hemos visto que dentro de las técnicas de filtrado colaborativo podemos distinguir entre las "basado en memoria" y las basdas en factorización de matrices.

En este notebook vamos a ver un ejemplo con Spark (pyspark) y este segundo tipo de sistemas

![](../img/memory-model-cf.jpg)

Para ello nos basamos en el siguiente [guión](https://databricks-training.s3.amazonaws.com/movie-recommendation-with-mllib.html) (ya han dado de baja la web pero podemos usar el siguiente [link](https://web.archive.org/web/20160316113725/https://databricks-training.s3.amazonaws.com/movie-recommendation-with-mllib.html)) del Spark Summit 2014.

### Antes de empezar

Necesitamos descargar los datos en la carpeta `../datos`

In [1]:
!ls -l ../datos/ml-1m

total 24328
-rw-r----- 1 jorge jorge   171308 mar 26  2003 movies.dat
-rw-r----- 1 jorge jorge 24594131 feb 28  2003 ratings.dat
-rw-r----- 1 jorge jorge     5577 ene 29  2016 README
-rw-r----- 1 jorge jorge   134368 feb 28  2003 users.dat


**NOTA:** Si no existen las carpetas `ml-1m` y `tag-genome` usamos el script `descargar_movilens.sh` para descargarlos.

## Los datos

En la carpeta `ml-1m`  que contiene: 

> Stable benchmark dataset. 1 million ratings from 6000 users on 4000 movies. Released 2/2003.

Hemos descargado estos datos que son pequeños para hacer las pruebas, pero el sistema que vamos a utilizar con Spark es distribuido y lo podríamos hacer sobre un cluster con el mismo código para datos más grandes.

Los datos que incluye MovieLens son:

* `movies.dat`: Incluye el catálogo de películas separado por `::` cada campo.
* `ratings.dat`: Incluye los ratings entre usuarios y películas en este caso la puntuación (de 1 a 5) que han dado a esa película. Este archivo es nuestra matriz $M_{(n, p)}$ .
* `users.dat`: Incluye información de los usuarios pero en nuestro ejercicio no vamos a utilizar este archivo.


In [2]:
!head ../datos/ml-1m/movies.dat

1::Toy Story (1995)::Animation|Children's|Comedy
2::Jumanji (1995)::Adventure|Children's|Fantasy
3::Grumpier Old Men (1995)::Comedy|Romance
4::Waiting to Exhale (1995)::Comedy|Drama
5::Father of the Bride Part II (1995)::Comedy
6::Heat (1995)::Action|Crime|Thriller
7::Sabrina (1995)::Comedy|Romance
8::Tom and Huck (1995)::Adventure|Children's
9::Sudden Death (1995)::Action
10::GoldenEye (1995)::Action|Adventure|Thriller


In [3]:
!head ../datos/ml-1m/ratings.dat

1::1193::5::978300760
1::661::3::978302109
1::914::3::978301968
1::3408::4::978300275
1::2355::5::978824291
1::1197::3::978302268
1::1287::5::978302039
1::2804::5::978300719
1::594::4::978302268
1::919::4::978301368


## Incluirnos en el recomendador

Una de las características importantes de los sitemas de recomendación basados en factorización de matrices. Es que desde el entrenamiento del modelo tendremos que incluir a todos los usuarios a los que vamos a querer recomendar. Al contrario que otros modelos de *machine learning* donde una vez entrenado el modelo podemos predecir a nuevos usuarios.

Para ello vamos a incluir nuestras preferencias como un nuevo usuario y después veremos las recomendaciones que obtenemos para nosotros mismos.

¿Cómo hacemos esto?

El siguiente script en python `spark_als/bin/rateMovies` sirve para generar nuestras recomendaciones.

Una vez ejecutado se crearán nuestros ratings en el archivo `personalRatings.txt`

In [4]:
!cat personalRatings.txt

0::1::1::1621347210
0::780::4::1621347210
0::590::1::1621347210
0::1210::5::1621347210
0::648::5::1621347210
0::344::1::1621347210
0::165::5::1621347210
0::153::5::1621347210
0::597::1::1621347210
0::1580::4::1621347210
0::231::1::1621347210


## Spark y MLlib

Para nuestro recomendador vamos a usar Spark y la librería MLlib que incluye el algoritmo ALS:    

&nbsp;<br>

![](../img/matrix_factorization.png)

Lo primero de todo comprobamos que tenemos creado el `SparkContext`:

In [5]:
import os
import sys
import pandas as pd
import numpy as np

from pyspark import SparkConf
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T

# Cargamos las funciones definidas en el archivo funciones_auxiliares.py
from funciones_auxiliares import *

In [6]:
conf = (

    SparkConf()
    .setAppName(u"Sistemas de Recomendación")
    .set("spark.executor.memory", "4g")
    .set("spark.executor.cores", "2")
    .set("spark.default.parallelism", "800")
    .set("spark.sql.shuffle.partitions", "800")
    .set("spark.submit.pyFiles", "funciones_auxiliares.py")

)

In [7]:
spark = (

    SparkSession.builder
    .config(conf=conf)
    .enableHiveSupport()
    .getOrCreate()

)

In [8]:
spark

Usamos la función `loadRatings` para cargar nuestros ratings personales: 

Definimos la carpeta donde se encuentras nuestros archivos y usamos la función `parseRating` de manera distribuida:

In [9]:
ratings_hdfs = '../datos/ml-1m/ratings.dat'

In [10]:
try:
    spark.read.options(header=False, sep="::").csv(ratings_hdfs).show()
except Exception as e: 
    print(e)

+---+----+---+---------+
|_c0| _c1|_c2|      _c3|
+---+----+---+---------+
|  1|1193|  5|978300760|
|  1| 661|  3|978302109|
|  1| 914|  3|978301968|
|  1|3408|  4|978300275|
|  1|2355|  5|978824291|
|  1|1197|  3|978302268|
|  1|1287|  5|978302039|
|  1|2804|  5|978300719|
|  1| 594|  4|978302268|
|  1| 919|  4|978301368|
|  1| 595|  5|978824268|
|  1| 938|  4|978301752|
|  1|2398|  4|978302281|
|  1|2918|  4|978302124|
|  1|1035|  5|978301753|
|  1|2791|  4|978302188|
|  1|2687|  3|978824268|
|  1|2018|  4|978301777|
|  1|3105|  5|978301713|
|  1|2797|  4|978302039|
+---+----+---+---------+
only showing top 20 rows



In [11]:
lines = spark.sparkContext.textFile(ratings_hdfs)

In [12]:
lines.take(4)

['1::1193::5::978300760',
 '1::661::3::978302109',
 '1::914::3::978301968',
 '1::3408::4::978300275']

In [13]:
parts = lines.map(lambda row: row.split("::"))

In [14]:
parts.take(4)

[['1', '1193', '5', '978300760'],
 ['1', '661', '3', '978302109'],
 ['1', '914', '3', '978301968'],
 ['1', '3408', '4', '978300275']]

In [15]:
ratingsRDD = (
    parts
    .map(lambda p: Row(userId=int(p[0]), movieId=int(p[1]), rating=float(p[2]), timestamp=int(p[3])))
)

In [16]:
ratingsRDD.take(4)

[Row(userId=1, movieId=1193, rating=5.0, timestamp=978300760),
 Row(userId=1, movieId=661, rating=3.0, timestamp=978302109),
 Row(userId=1, movieId=914, rating=3.0, timestamp=978301968),
 Row(userId=1, movieId=3408, rating=4.0, timestamp=978300275)]

In [17]:
ratingsRDD.count()

1000209

In [18]:
ratings = ratingsRDD.toDF()

In [19]:
ratings.show(4)

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|   1193|   5.0|978300760|
|     1|    661|   3.0|978302109|
|     1|    914|   3.0|978301968|
|     1|   3408|   4.0|978300275|
+------+-------+------+---------+
only showing top 4 rows



Pegamos ahora nuestros ratings al mismo `DF`:

In [20]:
myRatings = pd.read_csv(

    "personalRatings.txt",
    sep="::",
    names=["userId", "movieId", "rating", "timestamp"],
    engine='python'

)

In [21]:
myRatings

,userId,movieId,rating,timestamp
0,0,1,1,1621347210
1,0,780,4,1621347210
2,0,590,1,1621347210
3,0,1210,5,1621347210
4,0,648,5,1621347210
5,0,344,1,1621347210
6,0,165,5,1621347210
7,0,153,5,1621347210
8,0,597,1,1621347210
9,0,1580,4,1621347210


In [22]:
ratings = (

    ratings
    .unionByName(
        spark.createDataFrame(myRatings)
    )

).cache()

In [23]:
ratings.count()

1000220

In [24]:
movies_hdfs = '../datos/ml-1m/movies.dat'

In [25]:
movies = (

    spark.sparkContext
    .textFile(movies_hdfs)
    .map(lambda x: x.split("::"))
    .map(lambda x: Row(movieId=x[0], movieTitle=x[1], genres=x[2]))
    .toDF()

).cache()

In [26]:
conteos = (

    ratings
    .select(
        F.count("*").alias("count"),
        F.countDistinct('userId').alias('userId'),
        F.countDistinct('movieId').alias('movieId')
    )

).first()

Una vez cargados ambos archivos vamos a contar el número de películas, usuarios y ratings que tenemos:

In [27]:
print("Got %d ratings from %d users on %d movies." % conteos)

Got 1000220 ratings from 6041 users on 3706 movies.


Luego siguiendo nuestra notación tenemos que:

* $n=6041$
* $p=3706$

Así que la matriz $M$ tiene un tamaño de $6041\cdot3706=22387946$ pero solo tenemos información de $1000218$, es decir un 4%.

### Ejecución ALS

In [28]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

In [29]:
als = ALS(

    maxIter=5, 
    regParam=0.01, 
    userCol="userId", 
    itemCol="movieId", 
    ratingCol="rating",

)

In [30]:
model = als.fit(ratings)

In [31]:
model

ALSModel: uid=ALS_ab24aca09da7, rank=10

In [32]:
predictions = model.transform(ratings)

In [33]:
predictions.printSchema()

root
 |-- userId: long (nullable = true)
 |-- movieId: long (nullable = true)
 |-- rating: double (nullable = true)
 |-- timestamp: long (nullable = true)
 |-- prediction: float (nullable = false)



In [34]:
predictions.show()

+------+-------+------+----------+----------+
|userId|movieId|rating| timestamp|prediction|
+------+-------+------+----------+----------+
|  2122|   1580|   3.0| 974646450| 3.3192425|
|  3997|   1580|   5.0| 965579670|  4.535497|
|  1483|   1580|   3.0| 974752437| 3.2704625|
|  1721|   1580|   4.0| 974705774| 2.6476598|
|  2235|   1580|   3.0| 974613856|  4.141765|
|  4161|   1580|   3.0| 965340333| 2.5853975|
|  4219|   1580|   2.0| 965316151| 3.8178954|
|  1139|   1580|   4.0| 974878436| 3.4748604|
|  1322|   1580|   5.0| 974781764| 4.2643847|
|  5071|   1580|   4.0| 962458344|  4.163615|
|  5345|   1580|   4.0| 960671358|  4.006034|
|  2443|   1580|   2.0| 974218362| 2.9415839|
|  3089|   1580|   5.0| 969665122|  4.950528|
|  4239|   1580|   4.0| 965310412| 3.9529643|
|  4364|   1580|   5.0| 965184234| 4.0414877|
|  5074|   1580|   1.0| 962430441| 2.8988395|
|  5308|   1580|   4.0| 960934992|    3.5239|
|  4294|   1580|   3.0|1006400387| 2.7019787|
|  5217|   1580|   3.0| 961552591|

In [35]:
evaluator = RegressionEvaluator(
    metricName="rmse", 
    labelCol="rating",
    predictionCol="prediction"
)

In [36]:
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 0.7709347330811545


In [37]:
userRecs = model.recommendForAllUsers(10)

In [38]:
userRecs.printSchema()

root
 |-- userId: integer (nullable = false)
 |-- recommendations: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- movieId: integer (nullable = true)
 |    |    |-- rating: float (nullable = true)



In [39]:
userRecs.show()

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|  1580|[{3636, 7.8263874...|
|  2122|[{1471, 6.255006}...|
|  3175|[{2192, 8.515926}...|
|  5156|[{1930, 8.155917}...|
|  3997|[{1793, 8.16977},...|
|  3918|[{1450, 9.263441}...|
|  1829|[{960, 9.143938},...|
|  4519|[{2562, 7.8358164...|
|  1990|[{1872, 8.173045}...|
|  2580|[{811, 8.713287},...|
|  1721|[{3636, 4.7560596...|
|  4161|[{2602, 5.957645}...|
|  1483|[{682, 8.26465}, ...|
|  1025|[{1685, 9.928408}...|
|  2235|[{3860, 8.101308}...|
|  3179|[{3333, 8.597073}...|
|  4219|[{557, 6.3651156}...|
|  4929|[{572, 7.0321083}...|
|  5071|[{3003, 7.3115177...|
|  1322|[{3853, 6.248697}...|
+------+--------------------+
only showing top 20 rows



In [40]:
(

    userRecs
    .filter(""" userId = 0 """)
    .withColumn("recommendations",F.explode("recommendations"))
    .withColumn("movieId",F.col('recommendations')['movieId'])
    .withColumn("rating",F.col('recommendations')['rating'])
    .drop("recommendations")
    .join(movies, 'movieId')
    .orderBy(F.desc('rating'))
    
).toPandas()

,movieId,userId,rating,movieTitle,genres
0,3636,0,22.195744,Those Who Love Me Can Take the Train (Ceux qui...,Drama
1,1567,0,19.001030,"Last Time I Committed Suicide, The (1997)",Drama
2,811,0,15.260990,"Bewegte Mann, Der (1994)",Comedy
3,2483,0,15.165466,"Day of the Beast, The (El D�a de la bestia) (1...",Comedy|Horror|Thriller
4,3892,0,14.325734,Anatomy (Anatomie) (2000),Horror
5,2086,0,14.281295,One Magic Christmas (1985),Drama|Fantasy
6,2913,0,13.998003,"Mating Habits of the Earthbound Human, The (1998)",Comedy
7,1685,0,13.811930,"I Love You, I Love You Not (1996)",Romance
8,2586,0,13.680561,"Goodbye, Lover (1999)",Comedy|Crime|Thriller
9,3437,0,13.676436,Cool as Ice (1991),Drama


In [41]:
movieRecs = model.recommendForAllItems(10)

In [42]:
movieRecs.show()

+-------+--------------------+
|movieId|     recommendations|
+-------+--------------------+
|   1580|[{5072, 5.5607553...|
|   2122|[{2713, 6.291383}...|
|   3175|[{5072, 5.6118464...|
|   3918|[{1445, 7.6100197...|
|   1829|[{5901, 9.374512}...|
|   1990|[{5214, 7.814201}...|
|   2580|[{283, 5.6033506}...|
|   1721|[{1237, 6.198267}...|
|   1483|[{1102, 8.067711}...|
|   1025|[{4676, 6.5687475...|
|   2235|[{5258, 2.7450962...|
|   3179|[{41, 5.674091}, ...|
|   1322|[{0, 6.1293154}, ...|
|   1139|[{283, 5.2077594}...|
|   3220|[{5258, 2.7450962...|
|   2443|[{2441, 10.101035...|
|   2923|[{1445, 8.755112}...|
|     85|[{1098, 6.2206593...|
|   2525|[{1445, 4.884277}...|
|   3876|[{947, 8.872664},...|
+-------+--------------------+
only showing top 20 rows



In [43]:
users = ratings.select(als.getUserCol()).distinct().limit(3)

In [44]:
users.show()

+------+
|userId|
+------+
|  2040|
|  2927|
|  5409|
+------+



In [45]:
userSubsetRecs = model.recommendForUserSubset(users, 10)

In [46]:
userSubsetRecs.show()

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|  2040|[{2545, 11.509549...|
|  5409|[{2487, 9.187382}...|
|  2927|[{2342, 8.059692}...|
+------+--------------------+



In [47]:
userSubsetRecs.printSchema()

root
 |-- userId: integer (nullable = false)
 |-- recommendations: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- movieId: integer (nullable = true)
 |    |    |-- rating: float (nullable = true)



### Entrenamiento de los parámetros

Para decidir qué parámetros utilizar en nuestro algoritmo vamos a dividir la muestra en tres trozos:
entrenamiento (60%), validación (20%) y test (20%). Para ello lo hacemos basado en el último dígito del `timestamp` 
(ver la función `parseRating` línea 12)

In [48]:
(training, test) = ratings.randomSplit([0.8, 0.2], seed=1234)

Seleccionamos ahora los posibles valores de nuestros parámetros:

In [49]:
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [50]:
print(ALS().explainParams())

alpha: alpha for implicit preference (default: 1.0)
blockSize: block size for stacking input data in matrices. Data is stacked within partitions. If block size is more than remaining data in a partition then it is adjusted to the size of this data. (default: 4096)
checkpointInterval: set checkpoint interval (>= 1) or disable checkpoint (-1). E.g. 10 means that the cache will get checkpointed every 10 iterations. Note: this setting will be ignored if the checkpoint directory is not set in the SparkContext. (default: 10)
coldStartStrategy: strategy for dealing with unknown or new users/items at prediction time. This may be useful in cross-validation or production scenarios, for handling user/item ids the model has not seen in the training data. Supported values: 'nan', 'drop'. (default: nan)
finalStorageLevel: StorageLevel for ALS model factors. (default: MEMORY_AND_DISK)
implicitPrefs: whether to use implicit preference (default: False)
intermediateStorageLevel: StorageLevel for interme

In [51]:
als = ALS(

    userCol="userId", 
    itemCol="movieId", 
    ratingCol="rating",
    coldStartStrategy="drop"

)

In [52]:
paramGrid = (

    ParamGridBuilder()
    .addGrid(als.rank, [10, 100])
    .addGrid(als.regParam, [0.01, 0.1])
    .addGrid(als.maxIter, [5, 10])
    .build()

)

In [53]:
crossval = CrossValidator(

    estimator=als,
    estimatorParamMaps=paramGrid,
    evaluator=evaluator,
    numFolds=2

) 

**¡Cuidado esta parte tarda mucho!**

Un ejercicio habitual en *machine learning* es comparar el resultado de nuestro modelo con el *baseline*. En este caso con la media de los ratings y ver si nuestro modelo es mejor y en cuanto

### Modelo final
Terminamos entrenando el modelo final con todos los datos y los parámetros que hemos elegido

In [54]:
finalModel = ALS(

    regParam=0.01, #mallado.iloc[0]['regParam'],
    maxIter=20, #mallado.iloc[0]['maxIter'],
    rank=100, #mallado.iloc[0]['rank'],
    userCol="userId", 
    itemCol="movieId", 
    ratingCol="rating",
    coldStartStrategy="drop"

).fit(ratings)

In [55]:
rmse = evaluator.evaluate(finalModel.transform(ratings))
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 0.3635972557513914


## Ver nuestras recomendaciones

Vamos a recuperar las recomendaciones según los ratings que pusimos al principio

In [56]:
userRecs = finalModel.recommendForAllUsers(20)

¿Mejor?

In [57]:
recommendations = (

    userRecs
    .filter(""" userId = 0 """)
    .withColumn("recommendations", F.explode("recommendations"))
    .withColumn("movieId", F.col('recommendations')['movieId'])
    .withColumn("rating", F.col('recommendations')['rating'])
    .drop("recommendations")
    .join(ratings, ['userId', 'movieId'], 'left_anti')
    .join(movies, 'movieId')
    .orderBy(F.desc('rating'))

).toPandas()

In [58]:
recommendations

,movieId,userId,rating,movieTitle,genres
0,2628,0,5.048930,Star Wars: Episode I - The Phantom Menace (1999),Action|Adventure|Fantasy|Sci-Fi
1,327,0,4.624232,Tank Girl (1995),Action|Comedy|Musical|Sci-Fi
2,1196,0,4.471797,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Drama|Sci-Fi|War
3,592,0,4.380369,Batman (1989),Action|Adventure|Crime|Drama
4,1465,0,4.360483,Rosewood (1997),Drama
5,3786,0,4.215225,But I'm a Cheerleader (1999),Comedy
6,1377,0,4.195571,Batman Returns (1992),Action|Adventure|Comedy|Crime
7,3793,0,4.152361,X-Men (2000),Action|Sci-Fi
8,2115,0,4.139450,Indiana Jones and the Temple of Doom (1984),Action|Adventure
9,2297,0,4.107885,What Dreams May Come (1998),Drama|Romance


## Entendiendo cómo se realizan las predicciones

Vamos a entender qué descomposición se ha realizado y cómo se realizan las predicciones. 
La matriz de ratings tiene tamaño $(6041\times3706)$ como hemos visto y en el entrenamiento se ha decidido utilizar 100 variables latentes luego la descomposición que hemos realizado es:
&nbsp;<br>
&nbsp;<br>

$$
{\Large
M_{(6041\times 3706)} = U_{(6041\times 100)}\;V_{(100 \times 3706)}
}
$$

¿Dónde están esas matrices calculadas?

In [59]:
6041 *  3706

22387946

In [60]:
100 * (6041 +  3706)

974700

In [61]:
finalModel.itemFactors

DataFrame[id: int, features: array<float>]

In [62]:
finalModel.userFactors

DataFrame[id: int, features: array<float>]

In [63]:
len(finalModel.itemFactors.first()['features'])

100

In [64]:
len(finalModel.userFactors.first()['features'])

100

In [65]:
finalModel.itemFactors.count()

3706

In [66]:
finalModel.userFactors.count()

6041

Como hemos visto el objeto `finalModel` además contiene varias funciones para hacer las predicciones, pero vamos a hacerlo a mano para entender cómo funciona algebráicamente. 

Nueso id de usuario es el 0 así que podemos quedarnos con la fila de la matriz $U$ que hace referencia a nuestro usuario:

In [67]:
user_feature = np.array(finalModel.userFactors.filter(""" id==0 """).first()['features'])
user_feature

array([ 0.32979172,  0.36902419,  0.5841791 , -0.53383267,  0.22992346,
        0.34272107, -0.25297165, -0.23842192,  0.17236459,  0.09003194,
       -0.03180465,  0.18773952,  0.23166509, -0.14456429, -0.07406183,
        0.28490496, -0.25527915,  0.06423634, -0.29885006, -0.12479928,
        0.20707105, -0.22934406,  0.11501134, -0.09217511, -0.02077647,
        0.29334366,  0.30267996, -0.55899799,  0.3187871 , -0.07115725,
       -0.22504982,  0.11304507,  0.70787483,  0.07149173, -0.01693568,
       -0.27206326, -0.17124835, -0.15900284,  0.15195943,  0.16602218,
        0.08029862,  0.09966341,  0.09538151,  0.26488343,  0.41508421,
       -0.04243556, -0.32851589,  0.020768  , -0.07434284,  0.12231325,
       -0.14042556, -0.21324213, -0.08379763, -0.12333025, -0.28026763,
        0.0207203 , -0.75072771, -0.07026741,  0.42198446, -0.33457816,
        0.12093792,  0.20755197,  0.1624175 ,  0.08590674,  0.07004281,
       -0.62131387, -0.27873215,  0.06122407,  0.22594926, -0.12

Recuperamos nuestra primera recomendación:

In [68]:
recommendations.head(1)

,movieId,userId,rating,movieTitle,genres
0,2628,0,5.04893,Star Wars: Episode I - The Phantom Menace (1999),Action|Adventure|Fantasy|Sci-Fi


Extraemos de la columna $V$ la columna correspondiente con esta película

In [69]:
product_features = np.array(
    finalModel.itemFactors
    .filter(F.col('id') == int(recommendations.movieId[0]))
    .first()['features']
)
product_features

array([ 0.11333473,  0.37873894, -0.00250778, -0.30390206,  0.89733988,
        0.37543574, -0.33527046,  0.31515613, -0.16556831, -0.25413299,
        0.11919904,  0.07154579,  0.12066976, -0.09779519,  0.04108936,
        0.34447846, -0.60799289, -0.43334195, -0.38065094,  0.29641762,
       -0.38296163, -0.02719031, -0.04107076,  0.09960482, -0.37185633,
       -0.24337354,  0.54577518, -0.40692151,  0.30871862,  0.31748164,
        0.15196776,  0.19511372,  0.89528084,  0.26868919, -0.38777366,
       -0.03202303, -0.22717068, -0.16980377, -0.06075339,  0.85360372,
       -0.44064128,  0.19398573,  0.83698028,  0.00733232,  0.23501657,
        0.71009415,  0.12128538,  0.06582934,  0.53402829,  0.14035062,
       -0.548338  ,  0.09853847, -0.20872313,  0.54332489,  0.05802249,
       -0.13475439, -1.06770158, -0.35593227,  0.02224126,  0.33024111,
        0.2160878 ,  0.29653436, -0.00614211,  0.30231351, -0.07439789,
       -0.94687593, -0.20702834, -0.34044313,  0.74359393, -0.74

Para terminar, es fácil de comprobar que matemáticamente:
$$
{\Large
m_{ij} =\; <u_i, v_j>
}
$$

Es decir, el rating del usuario $i$ y el item $j$ es el producto escalar de la fila  $i$-esima de la matriz $U$ y la columna $j$-esima de la matriz $V$

In [70]:
np.dot(user_feature, product_features)

5.048931740154047

### Guardamos las dos matrices de manera local

Para terminar vamos a guardar las dos matrices de manera local para poder usarlas más tarde

In [71]:
item_factors = (

    finalModel
    .itemFactors

).toPandas()

In [72]:
item_factors.to_json('item_factors.json', orient='records')

In [73]:
user_factors = (

    finalModel
    .userFactors

).toPandas()

In [74]:
user_factors.to_json('user_factors.json', orient='records')

## Ejercicio

Crear un sistema de recomendaciones para los datos de **last.fm**

In [75]:
import os
import sys
import pandas as pd
import numpy as np

from pyspark import SparkConf
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T

# Cargamos las funciones definidas en el archivo funciones_auxiliares.py
from funciones_auxiliares import *

In [76]:
conf = (

    SparkConf()
    .setAppName(u"Sistemas de Recomendación")
    .set("spark.executor.memory", "4g")
    .set("spark.executor.cores", "2")
    .set("spark.default.parallelism", "800")
    .set("spark.sql.shuffle.partitions", "800")
    .set("spark.submit.pyFiles", "funciones_auxiliares.py")

)

In [77]:
spark = (

    SparkSession.builder
    .config(conf=conf)
    .enableHiveSupport()
    .getOrCreate()

)

<center>

![image](../img/logo_lastfm.png)
    
</center>

http://ocelma.net/MusicRecommendationDataset/lastfm-360K.html

https://musicbrainz.org

In [78]:
!head ../datos/lastfm-dataset-360K/usersha1-artmbid-artname-plays.tsv 

00000c289a1829a808ac09c00daf10bc3c4e223b	3bd73256-3905-4f3a-97e2-8b341527f805	betty blowtorch	2137
00000c289a1829a808ac09c00daf10bc3c4e223b	f2fb0ff0-5679-42ec-a55c-15109ce6e320	die Ärzte	1099
00000c289a1829a808ac09c00daf10bc3c4e223b	b3ae82c2-e60b-4551-a76d-6620f1b456aa	melissa etheridge	897
00000c289a1829a808ac09c00daf10bc3c4e223b	3d6bbeb7-f90e-4d10-b440-e153c0d10b53	elvenking	717
00000c289a1829a808ac09c00daf10bc3c4e223b	bbd2ffd7-17f4-4506-8572-c1ea58c3f9a8	juliette & the licks	706
00000c289a1829a808ac09c00daf10bc3c4e223b	8bfac288-ccc5-448d-9573-c33ea2aa5c30	red hot chili peppers	691
00000c289a1829a808ac09c00daf10bc3c4e223b	6531c8b1-76ea-4141-b270-eb1ac5b41375	magica	545
00000c289a1829a808ac09c00daf10bc3c4e223b	21f3573f-10cf-44b3-aeaa-26cccd8448b5	the black dahlia murder	507
00000c289a1829a808ac09c00daf10bc3c4e223b	c5db90c4-580d-4f33-b364-fbaa5a3a58b5	the murmurs	424
00000c289a1829a808ac09c00daf10bc3c4e223b	0639533a-0402-40ba-b6e0-18b067198b73	lunachicks	403


In [79]:
esquema = T.StructType([

    T.StructField('user_id',T.StringType(),True),
    T.StructField('artist_id',T.StringType(),True),
    T.StructField('artist_name',T.StringType(),True),
    T.StructField('plays',T.DoubleType(),True)

])

In [80]:
plays = (

    spark.read
    .csv('../datos/lastfm-dataset-360K/usersha1-artmbid-artname-plays.tsv', schema=esquema, sep='\t')
    .na.drop()

)

#### Pistas y consejos

Para agilizar el proceso, podemos usar una versión "pequeña" que se os facilita:

In [81]:
plays = spark.read.parquet('../datos/lastfm_mini.parquet')

In [82]:
plays.show()

+--------------------+--------------------+-----------+-----+
|             user_id|           artist_id|artist_name|plays|
+--------------------+--------------------+-----------+-----+
|8358173502fcfd30b...|382f1005-e9ab-468...|       2pac| 54.0|
|f434362a46b823536...|382f1005-e9ab-468...|       2pac|545.0|
|135750e1a2107ae20...|382f1005-e9ab-468...|       2pac| 54.0|
|41a072560f38290a2...|382f1005-e9ab-468...|       2pac|201.0|
|57da4f4db6fc10eb7...|382f1005-e9ab-468...|       2pac| 86.0|
|3f51326aec6893f15...|382f1005-e9ab-468...|       2pac| 95.0|
|ac4f934779afac87b...|382f1005-e9ab-468...|       2pac|163.0|
|858ebba231d365236...|382f1005-e9ab-468...|       2pac| 99.0|
|24efaa9b0170cda92...|382f1005-e9ab-468...|       2pac| 71.0|
|dd5b2d9f5d10d428d...|382f1005-e9ab-468...|       2pac| 18.0|
|7550b588c297f6792...|382f1005-e9ab-468...|       2pac| 47.0|
|662eee6265ed61f4a...|382f1005-e9ab-468...|       2pac|257.0|
|6e7acb2ec69628dc6...|382f1005-e9ab-468...|       2pac|388.0|
|45d4068

El siguiente código puede ser de utilidad para usar con el `ALS` visto:

In [83]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml.pipeline import Pipeline

conversores = [

    StringIndexer(inputCol='user_id', outputCol='user_idx'),
    StringIndexer(inputCol='artist_id', outputCol='artist_idx')

]

In [84]:
spark.stop()